In [1]:
# This script views collects all nightlight data 
# corresponding to the villages in a given CSV 
# 
# 'Cell' --> 'Run All' 


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import argparse
import os
import sys
import time
import pickle
import numpy as np
import pandas as pd
from six.moves import xrange  
import tensorflow as tf
import gdal
from matplotlib import pyplot as plt
import geotiling as gt

In [2]:
df_annos = pd.read_csv("../data/annos_csv/state24_paths_density_labels_13k_Feb26-NoOverlapDistrict.csv", index_col=0)
img_path = "/home/timhu/all_tif"
image_composite = gt.ImageComposite("../data/F182011.v4c_web.stable_lights.avg_vis.tif", 1)

print(image_composite)

In [3]:
def nightlight_curr_image(i, km_side):
    #lat = df_annos.loc[df_annos['pop_density_class'] == class_val].values[i][6]
    lat = df_annos.latitude[i]
    #lon = df_annos.loc[df_annos['pop_density_class'] == class_val].values[i][5]
    lon = df_annos.longitude[i]
    
    col, row = image_composite.geoprops.lonlat2colrow(lon, lat)
#     print(image_composite.geoprops.xOrigin)
#     print(col, row)
    #FACT: true same size would be 4.5 by 4.5 pixels since each pixel is about a km2
    x = image_composite.gdal_dataset.ReadAsArray(col - int(km_side/2), row - int(km_side/2), km_side, km_side)
    np.place(x, x == -2147483647, 0)
    #print(x)
    return x

In [4]:
import numpy as np
import pandas as pd
import re,os,time, math, gdal
from PIL import Image

#df = pd.read_csv('/home/timhu/dfd-pop/data/annos_csv/state24_paths_density_labels_13k_Feb10.csv')

NL_DIR = "/home/timhu/nl_jpg/"
Wnl = Hnl = 5 
NL_sums = []
pop_densities = []

def jpg_convert_save(tif_path, crop_width, crop_height, i):
    jpg_path = re.sub('tif', 'jpg', tif_path)
    jpg_path = re.sub('l8', 'nl', jpg_path)
    jpg_path = re.sub('500x500', str(crop_width)+'x'+str(crop_height), jpg_path)

    img_array = nightlight_curr_image(i, Wnl)
    im = Image.fromarray(img_array)
    im.save(os.path.join(NL_DIR, jpg_path), format='JPEG')
    

def get_NL_info(i):
    img_array = nightlight_curr_image(i, Wnl)
    a = img_array.flatten()
    return sum(a)/25

In [5]:
NL_sums = []
pop_densities = []

# @JAY this is the main data loop - NL sums are the input & pop densities are corresponding labels
for i, row in df_annos.iterrows():
    village_id = df_annos.village_id[i]
    pop_density = df_annos.pop_density_log2[i]
    #jpg_convert_save(l8_path, Wnl, Hnl, i) 
    nl_val = get_NL_info(i)
    NL_sums.append(nl_val)
    pop_densities.append(pop_density)
    
print(len(NL_sums))
print(len(pop_densities))

12678
12678


In [6]:
# plt.scatter(NL_sums, pop_densities,  color='black')

In [7]:
# # License: BSD 3 clause


# import matplotlib.pyplot as plt
# import numpy as np
# from sklearn import datasets, linear_model
# from sklearn.metrics import mean_squared_error, r2_score

# NL_sums = np.asarray(NL_sums).reshape(-1, 1)
# pop_densities = np.asarray(pop_densities).reshape(-1, 1)

# # # Use only one feature
# # diabetes_X = diabetes.data[:, np.newaxis, 2]

# # Split the data into training/testing sets
# X_train = NL_sums[:-2000]
# X_test = NL_sums[-2000:]

# # Split the targets into training/testing sets
# y_train = pop_densities[:-2000]
# y_test = pop_densities[-2000:]

# # Create linear regression object
# regr = linear_model.LinearRegression()

# # Train the model using the training sets
# regr.fit(X_train, y_train)

# # Make predictions using the testing set
# y_pred = regr.predict(X_test)

# # The coefficients
# print('Coefficients: \n', regr.coef_)
# # The mean squared error
# print("Mean squared error: %.2f"
#       % mean_squared_error(y_test, y_pred))
# # Explained variance score: 1 is perfect prediction
# print('r2 score: %.2f' % r2_score(y_test, y_pred))

# # Plot outputs
# plt.scatter(X_test, y_test,  color='black')
# plt.plot(X_test, y_pred, color='blue', linewidth=3)
# plt.xlabel('Max Nightlight Intensity')
# plt.ylabel('Log2 Popuation Density')
# # plt.xticks(())
# # plt.yticks(())

# plt.show()

In [8]:
# y = pop_densities
# predicted = NL_sums
# fig, ax = plt.subplots()
# ax.scatter(y, predicted, edgecolors=(0, 0, 0))
# ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
# ax.set_xlabel('Measured')
# ax.set_ylabel('Predicted')
# plt.show()
# print(sum(NL_sums)/12678)
# print(sum(pop_densities)/12678)